In [ ]:
!pip install selenium
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import numpy as np
import chromedriver_autoinstaller  # setup chrome options

In [ ]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

chromedriver_autoinstaller.install()  # set the target URL

#지원할 직무 선택

In [ ]:
job_categories = {
    "기획·전략": ["경영·비즈니스기획", "웹기획", "마케팅기획", "PL·PM·PO", "컨설턴트", "CEO·COO·CTO", "AI기획자", "AI사업전략"],
    "법무·사무·총무": ["경영지원", "사무담당자", "총무", "사무보조", "법무담당자", "비서", "변호사", "법무사", "변리사", "노무사", "AI윤리전문가"],
    "인사·HR": ["인사담당자", "HRD·HRM", "노무관리자", "잡매니저", "헤드헌터", "직업상담사"],
    "회계·세무": ["회계담당자", "경리", "세무담당자", "재무담당자", "감사", "IR·공시", "회계사", "세무사", "관세사"],
    "마케팅·광고·MD": ["AE(광고기획자)", "브랜드마케터", "퍼포먼스마케터", "CRM마케터", "온라인마케터", "콘텐츠마케터", "홍보", "설문·리서치", "MD", "카피라이터", "크리에이티브디렉터", "채널관리자", "그로스해커"],
    "AI·개발·데이터": ["백엔드개발자", "프론트엔드개발자", "웹개발자", "앱개발자", "시스템엔지니어", "네트워크엔지니어", "DBA", "데이터엔지니어", "데이터사이언티스트", "보안엔지니어", "소프트웨어개발자", "게임개발자", "하드웨어개발자", "AI/ML엔지니어", "블록체인개발자", "클라우드엔지니어", "웹퍼블리셔", "IT컨설팅", "QA", "AI/ML연구원", "데이터분석가", "데이터라벨러", "프롬프트엔지니어", "AI보안전문가", "MLOps엔지니어", "AI서비스개발자"],
    "디자인": ["그래픽디자이너", "3D디자이너", "제품디자이너", "산업디자이너", "광고디자이너", "시각디자이너", "영상디자이너", "웹디자이너", "UI·UX디자이너", "패션디자이너", "편집디자이너", "실내디자이너", "공간디자이너", "캐릭터디자이너", "환경디자이너", "아트디렉터", "일러스트레이터"],
    "물류·무역": ["물류관리자", "구매관리자", "자재관리자", "유통관리자", "무역사무원"],
    "운전·운송·배송": ["납품·배송기사", "배달기사", "수행·운전기사", "화물·중장비기사", "버스기사", "택시기사", "조종·기관사"],
    "영업": ["제품영업", "서비스영업", "해외영업", "광고영업", "금융영업", "법인영업", "IT·기술영업", "영업관리", "영업지원"],
    "고객상담·TM": ["인바운드상담원", "아웃바운드상담원", "고객센터관리자"],
    "금융·보험": ["금융사무", "보험설계사", "손해사정사", "심사", "은행원·텔러", "계리사", "펀드매니저", "애널리스트"],
    "식·음료": ["요리사", "조리사", "제과제빵사", "바리스타", "셰프·주방장", "카페·레스토랑매니저", "홀서버", "주방보조", "소믈리에·바텐더", "영양사", "식품연구원", "푸드스타일리스트"],
    "고객서비스·리테일": ["설치·수리기사", "정비기사", "호텔종사자", "여행에이전트", "매장관리자", "뷰티·미용사", "애견미용·훈련", "안내데스크·리셉셔니스트", "경호·경비", "운영보조·매니저", "이벤트·웨딩플래너", "주차·주유원", "스타일리스트", "장례지도사", "가사도우미", "승무원", "플로리스트"],
    "엔지니어링·설계": ["전기·전자엔지니어", "기계엔지니어", "설계엔지니어", "설비엔지니어", "반도체엔지니어", "화학엔지니어", "공정엔지니어", "하드웨어엔지니어", "통신엔지니어", "RF엔지니어", "필드엔지니어", "R&D·연구원", "AI로봇엔지니어"],
    "제조·생산": ["생산직종사자", "생산·공정관리자", "품질관리자", "포장·가공담당자", "공장관리자", "용접사"],
    "교육": ["유치원·보육교사", "학교·특수학교교사", "대학교수·강사", "학원강사", "외국어강사", "기술·전문강사", "학습지·방문교사", "학원상담·운영", "교직원·조교", "교재개발·교수설계", "AI교육컨설턴트"],
    "건축·시설": ["건축가", "건축기사", "시공기사", "전기기사", "토목기사", "시설관리자", "현장관리자", "안전관리자", "공무", "소방설비", "현장보조", "감리원", "도시·조경설계", "환경기사", "비파괴검사원", "공인중개사", "감정평가사", "분양매니저"],
    "의료·바이오": ["의사", "한의사", "간호사", "간호조무사", "약사·한약사", "의료기사", "수의사", "수의테크니션", "병원코디네이터", "원무행정", "기타의료종사자", "의료·약무보조", "바이오·제약연구원", "임상연구원"],
    "미디어·문화·스포츠": ["PD·감독", "포토그래퍼", "영상편집자", "사운드엔지니어", "스태프", "출판·편집", "배급·제작자", "콘텐츠에디터", "크리에이터", "기자", "작가", "아나운서", "리포터·성우", "MC·쇼호스트", "모델", "연예인·매니저", "인플루언서", "통번역사", "큐레이터", "음반기획", "스포츠강사", "AI콘텐츠크리에이터"],
    "공공·복지": ["사회복지사", "요양보호사", "환경미화원", "보건관리자", "사서", "자원봉사자", "방역·방재기사"]
}

# 데이터프레임 생성
data = []
for category, jobs in job_categories.items():
    for job in jobs:
        data.append({"대분류": category, "직무명": job})

df = pd.DataFrame(data)

In [ ]:
for i in range(len(job_categories)):
  print(f"{i+1}. {list(job_categories.keys())[i]}")
arr1 = int(input('지원한 카테고리를 선택하세요 : '))
category = list(job_categories.keys())[arr1-1]
print()
for i in range(len(job_categories[list(job_categories.keys())[arr1-1]])):
  print(f"{i+1}. {job_categories[list(job_categories.keys())[arr1-1]][i]}")
arr2 = int(input('지원한 직무를 선택하세요 : '))
job = job_categories[list(job_categories.keys())[arr1-1]][arr2-1]
category_code = 10026 + arr1 - 1

#자소서 주소 목록

In [ ]:
#==============================합격 자소서 주소========================================
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
assay = []

# Chrome 설정
options = Options()
options.add_argument('--headless')  # 창 안 띄우기
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)
for i in range(1,2):
  driver.get(f"https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&schPart={category_code}&isSaved=1&Page={i}")
  # JS 렌더링 기다리기
  time.sleep(3)

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  # 자소서 목록 가져오기
  ul = soup.find("ul", class_="selfLists")
  li_list = ul.find_all("li")

  # #페이지 당 20개
  for i in range(len(li_list)):
    tag = li_list[i].find("a", class_="logo")
    first_li = li_list[i]
    link_tag = first_li.find("a", class_="logo")
    # 만약 a 태그가 없다면 (예외 처리)
    if not link_tag:
        continue
    href = tag['href']
    #href = link_tag.get("href", "")
    assay.append("https://www.jobkorea.co.kr" + href)
driver.quit()

In [ ]:
assay

#지원기업 / 지원년도 / 지원직무 / 질문 / 답변

In [ ]:
data = []
driver = webdriver.Chrome(options=chrome_options)
for i in range(len(assay)):
  question_list = []
  answer_list = []
  #링크 전달
  driver.get(assay[i])

  assay_info = driver.find_element(By.CSS_SELECTOR, 'div.selfTopBx').text
  lines = assay_info.splitlines()
  #===================== #지원기업 / 지원년도 / 지원직무
  company = lines[0]  # 금융결제원
  title = lines[2]    # 2022년 하반기 신입 백엔드개발자합격자소서
  title_words = title.split()
  year = title_words[0]
  apply_for = title_words[-1]

  #====================== #자소서 항목(질문)
  question_elements = driver.find_elements(By.CSS_SELECTOR, "dt")
  for elem in question_elements:
    if elem.text[0:2] == '질문':
      question_list.append(elem.text[7:len(elem.text)-2])
  #====================== #자소서 내용(답변)
  # JavaScript로 모든 dd에 class="show" 추가
  script = """
  const dds = document.querySelectorAll("dd");
  dds.forEach(dd => dd.classList.add("show"));
  """
  driver.execute_script(script)
  time.sleep(0.5)
  answer_list = []
  # 이제 모든 답변 div.tx 가져오기
  answer_elements = driver.find_elements(By.CSS_SELECTOR, "div.tx")
  answer_list = [elem.text.strip() for elem in answer_elements]
  answer_list = answer_list[0:len(question_list)]

  # 데이터 저장
  data.append({
      "회사": company,
      "연도": year,
      "직무": apply_for[:-5],
      "질문": question_list,
      "답변": answer_list,
      "링크": assay[i]
  })


driver.quit()

##직무에 맞는 데이터만 남기기

In [ ]:
data = [d for d in data if d['직무'] == job]

#사용자 자소서

##사용자 자소서 입력

In [ ]:
from google.colab import files

# 업로드 창이 뜨고, 파일 선택하면 업로드됨
uploaded = files.upload()

In [ ]:
file_path = '백엔드 자소서 샘플.pdf'

##사용자 자소서 텍스트 추출

In [ ]:
!pip install pymupdf

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [ ]:
pdf_text = extract_text_from_pdf(file_path)

In [ ]:
pdf_text[:100]

##텍스트 내용별 분류

In [ ]:
from gradio_client import Client

# Space 주소 (user/space_name)
client = Client("amd/gpt-oss-120b-chatbot")

# pdf_text: 이력서 PDF에서 추출한 원본 텍스트 (정리 안 된 상태)


# 챗봇에 보낼 프롬프트 작성
prompt = f"""아래는 이력서 원본 텍스트입니다.
내용별로 원본 내용을 최대한 유지한 상태로 분류해서 정리해 주세요.
각 항목은 '항목명', '내용'으로 구성된 표 형식으로 작성해 주세요.
결과는 파이썬 pandas DataFrame에 넣을 수 있도록 아래 형식처럼 JSON 형태의 리스트로 출력해 주세요:

예시:
```json
[
  {{ "항목": "이름", "내용": "홍길동" }},
  {{ "항목": "생년월일", "내용": "1990-01-01" }},
]
```

=== 이력서 원본 텍스트 ===
{pdf_text}
"""

# Space 호출
output = client.predict(prompt)

print("분류 및 정리 결과:")
print(output)


##분류된 내용 데이터화

In [ ]:
import re
import ast
import pandas as pd
import json

In [ ]:
# JSON 코드블록 추출
match = re.search(r"```json\s*(\[\s*{.*?}\s*\])\s*```", output, re.DOTALL)

if match:
    json_text = match.group(1).strip()
    try:
        user_data = json.loads(json_text)
        user_df = pd.DataFrame(user_data)
        print(user_df)
    except json.JSONDecodeError as e:
        print("JSON 파싱 오류:", e)
else:
    print("JSON 코드블록을 찾지 못했습니다.")

In [ ]:
user_df

##질문-답변만 추출

In [ ]:
user_qa = []

for i in range(len(user_df['항목'])):
    if len(user_df['내용'][i]) > 100:
        user_qa.append({
            'question': user_df['항목'][i],
            'anwer': user_df['내용'][i]
        })
user_qa

##비슷한 맥락의 질문 분류

In [ ]:
pip install konlpy

In [ ]:
from sentence_transformers import SentenceTransformer, util

# 1) 모델 불러오기 (한국어 RoBERTa 모델 추천)
model = SentenceTransformer('jhgan/ko-sroberta-multitask')


In [ ]:
# user_qa가 리스트 형태일 때
user_questions = [qa['question'] for qa in user_qa]
user_answers = [qa['anwer'] for qa in user_qa]  # 'anwer' 키 주의

pass_questions = []
pass_answers = []

for d in data:
    pass_questions.extend(d['질문'])
    pass_answers.extend(d['답변'])

user_embeds = model.encode(user_questions, convert_to_tensor=True)
pass_embeds = model.encode(pass_questions, convert_to_tensor=True)
cos_sim = util.cos_sim(user_embeds, pass_embeds)

##1.가장 유사한 자소서 하나만

In [ ]:
import pandas as pd

# user_qa가 리스트 형태일 때
user_questions = [qa['question'] for qa in user_qa]
user_answers = [qa['anwer'] for qa in user_qa]  # 'anwer' 키 주의

pass_questions = []
pass_answers = []

for d in data:
    pass_questions.extend(d['질문'])
    pass_answers.extend(d['답변'])

user_embeds = model.encode(user_questions, convert_to_tensor=True)
pass_embeds = model.encode(pass_questions, convert_to_tensor=True)
cos_sim = util.cos_sim(user_embeds, pass_embeds)

matched_data = []

for i, user_q in enumerate(user_questions):
    best_idx = cos_sim[i].argmax()
    score = cos_sim[i][best_idx].item()

    if score >= 0.5:
        matched_data.append({
            '사용자 질문': user_q,
            '사용자 답변': user_answers[i],
            '합격 질문': pass_questions[best_idx],
            '합격 답변': pass_answers[best_idx],
            '유사도': score
        })
    else:
        matched_data.append({
            '사용자 질문': user_q,
            '사용자 답변': user_answers[i],
            '합격 질문': None,
            '합격 답변': None,
            '유사도': score
        })

df_matched = pd.DataFrame(matched_data)
print(df_matched)


###바로 LLM 최종 피드백

In [ ]:
import time
import pandas as pd
from gradio_client import Client

# HuggingFace Space 클라이언트
client = Client("amd/gpt-oss-120b-chatbot")

feedback_results = []

# df_matched의 각 행마다 피드백 생성
for idx, row in df_matched.iterrows():
    # 합격 질문/답변이 없는 경우 건너뛰기
    if pd.isna(row['합격 질문']) or pd.isna(row['합격 답변']):
        feedback_results.append({
            '사용자 질문': row['사용자 질문'],
            '사용자 답변': row['사용자 답변'],
            '합격 질문': None,
            '합격 답변': None,
            '유사도': row['유사도'],
            '피드백': None
        })
        continue

    # 프롬프트 작성
    prompt = f"""
    아래는 사용자 자소서 질문과 답변, 그리고 유사한 합격 자소서 질문과 답변입니다.
    당신은 합격 자소서 내용을 참고하여 사용자 자소서에 대해 구체적이고 건설적인 피드백을 작성해주세요.

    사용자 자소서 질문: {row['사용자 질문']}
    사용자 자소서 답변: {row['사용자 답변']}

    합격 자소서 질문: {row['합격 질문']}
    합격 자소서 답변: {row['합격 답변']}

    사용자 자소서를 더 잘 다듬을 수 있도록 도움말을 작성해 주세요.
    """

    try:
        output = client.predict(prompt)
    except Exception as e:
        output = f"피드백 생성 실패: {e}"

    # 결과 저장
    feedback_results.append({
        '사용자 질문': row['사용자 질문'],
        '사용자 답변': row['사용자 답변'],
        '합격 질문': row['합격 질문'],
        '합격 답변': row['합격 답변'],
        '유사도': row['유사도'],
        '피드백': output
    })

    # API 요청 간 딜레이 (세션 다운 방지)
    time.sleep(2)

# 피드백 결과 DataFrame
df_feedback = pd.DataFrame(feedback_results)
print(df_feedback)

# CSV 저장
#df_feedback.to_csv("feedback_results.csv", index=False, encoding="utf-8-sig")


In [ ]:
print(df_feedback['사용자 질문'][0])
print(df_feedback['사용자 답변'][0])
print(df_feedback['피드백'][0])

##2.유사도 이상인 모든 자소서

In [ ]:
matched_data = []

threshold = 0.5  # 유사도 기준

for i, user_q in enumerate(user_questions):
    # 사용자 질문과 모든 합격 질문의 유사도 리스트
    sims = cos_sim[i]

    # 기준 이상인 인덱스 찾기
    matched_indices = [j for j, score in enumerate(sims) if score >= threshold]

    # 매칭 질문이 없으면 None 저장
    if not matched_indices:
        matched_data.append({
            '사용자 질문': user_q,
            '사용자 답변': user_answers[i],
            '합격 질문': None,
            '합격 답변': None,
            '유사도': None
        })
    else:
        # 여러 개 있으므로 각각 따로 저장
        for idx in matched_indices:
            matched_data.append({
                '사용자 질문': user_q,
                '사용자 답변': user_answers[i],
                '합격 질문': pass_questions[idx],
                '합격 답변': pass_answers[idx],
                '유사도': sims[idx].item()
            })

df_matched = pd.DataFrame(matched_data)
df_matched


### 그룹별로 '유사도' 기준 내림차순 정렬 후 상위 3개만

In [ ]:
# 그룹별로 '유사도' 기준 내림차순 정렬 후 상위 5개만
grouped = (
    df_matched
    .sort_values('유사도', ascending=False)  # 우선 전체에서 정렬
    .groupby(['사용자 질문', '사용자 답변'], group_keys=False)
    .apply(lambda g: g.nlargest(3, '유사도'))  # 각 그룹별 top 5
    .groupby(['사용자 질문', '사용자 답변'])
    .agg({
        '합격 질문': lambda x: list(x.dropna()),
        '합격 답변': lambda x: list(x.dropna()),
        '유사도': lambda x: list(x.dropna())
    })
    .reset_index()
)

###모델 하나로 전부 평가->요약하기

In [ ]:
from gradio_client import Client
import time
# Space 주소 (user/space_name)
client = Client("amd/gpt-oss-120b-chatbot")

final_feedbacks = []  # 각 사용자 질문에 대한 최종 피드백 저장

# 전체 사용자 질문 반복
for uq_idx in range(len(grouped['사용자 질문'])):
    user_question = grouped['사용자 질문'][uq_idx]
    user_answer = grouped['사용자 답변'][uq_idx]

    feedbacks = []  # 현재 질문에 대한 모든 피드백 저장

    # 해당 사용자 질문에 매칭된 모든 합격 자소서 질문/답변과 비교
    for pq_idx in range(len(grouped['합격 질문'][uq_idx])):
        pass_question = grouped['합격 질문'][uq_idx][pq_idx]
        pass_answer = grouped['합격 답변'][uq_idx][pq_idx]

        prompt = f"""
        아래는 사용자 자소서 질문과 답변, 그리고 유사한 합격 자소서 질문과 답변입니다.
        당신은 합격 자소서 내용을 참고하여 사용자 자소서에 대해 구체적이고 건설적인 피드백을 작성해주세요.

        사용자 자소서 질문: {user_question}
        사용자 자소서 답변: {user_answer}

        합격 자소서 질문: {pass_question}
        합격 자소서 답변: {pass_answer}

        사용자 자소서를 더 잘 다듬을 수 있도록 도움말을 작성해 주세요.
        """

        output = client.predict(prompt)
        feedbacks.append(output)
        time.sleep(2)
    # 각 피드백을 요약
    summary_prompt = f"""
    아래는 동일한 사용자 자소서 질문에 대해 여러 합격 자소서 내용을 참고해 생성한 피드백들입니다.
    이 피드백들에서 공통적으로 나타나는 핵심적인 조언과 내용을 요약해 주세요.
    요약문은 간결하고 명확하게 작성해 주세요.

    피드백들:
    {chr(10).join(feedbacks)}

    요약:
    """
    summary_output = client.predict(summary_prompt)

    # 최종 리스트에 저장
    final_feedbacks.append({
        "질문": user_question,
        "최종 피드백": summary_output
    })

# 모든 결과 출력
for item in final_feedbacks:
    print(f"질문: {item['질문']}")
    print(f"최종 피드백: {item['최종 피드백']}\n")
